In [1]:
import os
import numpy as np
import pandas as pd

from typing import List, Set, Dict, Optional

from kbc_pul.project_info import data_dir

from artificial_bias_experiments.evaluation.confidence_comparison.df_utils import ColumnNamesInfo
from kbc_pul.experiments_utils.load_df_ground_truth import get_df_ground_truth
from artificial_bias_experiments.noisy_prop_scores.scar.experiment_info import \
    NoisyPropScoresSCARExperimentInfo
from artificial_bias_experiments.noisy_prop_scores.scar.image_generation.load_rule_wrappers import \
    NoisyPropScoresDataFrames, load_noisy_prop_scores_scar_rule_wrapper_dfs
from artificial_bias_experiments.noisy_prop_scores.scar.noisy_prop_scores_scar_file_naming import \
    NoisyPropScoresSCARFileNamer
from kbc_pul.project_info import project_dir as kbc_e_metrics_project_dir

from kbc_pul.confidence_naming import ConfidenceEnum
from pylo.language.lp import Clause as PyloClause

# Noisy SCAR fancy table generation

In [2]:
dataset_name: str = "yago3_10"

true_label_frequency_list = [0.3, .7]

# true_prop_scores = PropScoresTwoSARGroups(
#     in_filter=true_prop_score_in_filter,
#     other=true_prop_score_other
# )

available_label_frequency_list: List[float] = [0.1, 0.2, .3, .4, .5, .6, .7, .8, .9, 1]
filename_ground_truth_dataset: str = os.path.join(
    data_dir, dataset_name, 'cleaned_csv', 'train.csv'
)
separator_ground_truth_dataset = "\t"
df_ground_truth: pd.DataFrame = get_df_ground_truth(filename_ground_truth_dataset, separator_ground_truth_dataset)
target_relation_list: List[str] = list(sorted(df_ground_truth["Rel"].unique()))

is_pca_version: bool = False

In [3]:
df_rule_wrapper_list: List[pd.DataFrame] = []

for true_label_frequency in true_label_frequency_list:
    for target_relation in target_relation_list:
        try:
            experiment_info=NoisyPropScoresSCARExperimentInfo(
                dataset_name=dataset_name,
                target_relation=target_relation,
                true_label_frequency=true_label_frequency,
                available_label_frequency_list=available_label_frequency_list,
                is_pca_version=is_pca_version
            )


            root_dir_specific_experiment_settings: str = NoisyPropScoresSCARFileNamer.get_dir_experiment_specific(
                experiment_info=experiment_info
            )

            noisy_prop_scores_dfs: NoisyPropScoresDataFrames = load_noisy_prop_scores_scar_rule_wrapper_dfs(
                experiment_dir=root_dir_specific_experiment_settings
            )

            df_rule_wrappers: pd.DataFrame = noisy_prop_scores_dfs.rule_wrappers
            df_rule_wrappers["target_relation"] = pd.Series(
                data=[target_relation] * len(df_rule_wrappers), index=df_rule_wrappers.index
            )

            df_noisy_prop_scores_to_metrics_map: pd.DataFrame = noisy_prop_scores_dfs.noisy_prop_scores_to_pu_metrics_map


            df_rule_wrappers:pd.DataFrame = df_rule_wrappers.merge(
                df_noisy_prop_scores_to_metrics_map,
                left_on=["Rule", "random_trial_index", "true_label_frequency"],
                right_on=["Rule", "random_trial_index", "true_label_frequency"],
            )
            df_rule_wrapper_list.append(df_rule_wrappers)
        except Exception as err:
            print(target_relation)
            print(err)

df_rule_wrappers_all_targets: pd.DataFrame = pd.concat(df_rule_wrapper_list, axis=0)
df_rule_wrappers_all_targets.head()

edited
No objects to concatenate
hasacademicadvisor
No objects to concatenate
hasmusicalrole
No objects to concatenate
haswebsite
No objects to concatenate
influences
No objects to concatenate
isknownfor
No objects to concatenate
playsfor
[Errno 2] No such file or directory: '/home/joschout/Documents/Repos/KUL-PUL/data/artificial_bias_experiments/noisy_prop_scores/scar/yago3_10/playsfor/not_pca/c0.3/rule_wrappers/pu_metrics/playsfor(A,B) :- isaffiliatedto(A,B)_trial9.tsv.gz'
wrotemusicfor
No objects to concatenate
edited
No objects to concatenate
hasacademicadvisor
No objects to concatenate
hasmusicalrole
No objects to concatenate
haswebsite
No objects to concatenate
influences
No objects to concatenate
isknownfor
No objects to concatenate
wrotemusicfor
No objects to concatenate


,true_label_frequency,random_trial_index,Rule,Nb supported predictions,Body size,$conf$,CWA,$\frac{\left| \mathbf{R}\right|}{\left| \mathbf{R_s}\right|} conf$ $p$,PCA $p$,$\frac{\left| \mathbf{R}\right|}{\left| \mathbf{R_s}\right|} conf$ $p^{-1}$,PCA ${p^{-1}}$,target_relation,noisy_label_frequency,IPW,IPW-PCA $p$,IPW-PCA ${p^{-1}}$,ICW
0,0.3,8,"actedin(A,B) :- directed(A,B)",165,5481,0.101806,0.030104,0.347231,0.166163,0.104007,0.040352,actedin,0.1,0.301040,0.166163,0.166163,0.301040
1,0.3,8,"actedin(A,B) :- directed(A,B)",165,5481,0.101806,0.030104,0.347231,0.166163,0.104007,0.040352,actedin,0.2,0.150520,0.166163,0.166163,0.150520
2,0.3,8,"actedin(A,B) :- directed(A,B)",165,5481,0.101806,0.030104,0.347231,0.166163,0.104007,0.040352,actedin,0.3,0.100347,0.166163,0.166163,0.100347
3,0.3,8,"actedin(A,B) :- directed(A,B)",165,5481,0.101806,0.030104,0.347231,0.166163,0.104007,0.040352,actedin,0.4,0.075260,0.166163,0.166163,0.075260
4,0.3,8,"actedin(A,B) :- directed(A,B)",165,5481,0.101806,0.030104,0.347231,0.166163,0.104007,0.040352,actedin,0.5,0.060208,0.166163,0.166163,0.060208


In [4]:
df_rule_wrappers_all_targets.columns

Index(['true_label_frequency', 'random_trial_index', 'Rule',
       'Nb supported predictions', 'Body size', '$conf$', 'CWA',
       '$\frac{\left| \mathbf{R}\right|}{\left| \mathbf{R_s}\right|} conf$ $p$',
       'PCA $p$',
       '$\frac{\left| \mathbf{R}\right|}{\left| \mathbf{R_s}\right|} conf$ $p^{-1}$',
       'PCA ${p^{-1}}$', 'target_relation', 'noisy_label_frequency', 'IPW',
       'IPW-PCA $p$', 'IPW-PCA ${p^{-1}}$', 'ICW'],
      dtype='object')

In [5]:
all_columns: List[str] = [
    'target_relation',
    'true_label_frequency',
    'noisy_label_frequency',
    'Rule',
    'random_trial_index',
    'Nb supported predictions',
    'Body size',
    '$conf$',
    'CWA',
    '$f^{*} \cdot conf$ (S)', 'PCA(S)', '$f^{*} \cdot conf$ (O)', 'PCA(O)',
        'IPW', 'IPW-PCA(S)', 'IPW-PCA(O)', 'ICW'
]
column_names_logistics =  [
    'target_relation',
    'true_label_frequency',
    'noisy_label_frequency',
    'Rule',
    'random_trial_index',
]
columns_to_select = column_names_logistics + [
    col for col in set([conf.value for conf in ConfidenceEnum])
]

## Drop body size & nb of predictions + reorder

In [6]:
df_rule_wrappers_all_targets = df_rule_wrappers_all_targets[columns_to_select]
df_rule_wrappers_all_targets.head()


,target_relation,true_label_frequency,noisy_label_frequency,Rule,random_trial_index,ICW,$conf$,$\frac{\left| \mathbf{R}\right|}{\left| \mathbf{R_s}\right|} conf$ $p^{-1}$,$\frac{\left| \mathbf{R}\right|}{\left| \mathbf{R_s}\right|} conf$ $p$,PCA ${p^{-1}}$,IPW,IPW-PCA $p$,IPW-PCA ${p^{-1}}$,CWA,PCA $p$
0,actedin,0.3,0.1,"actedin(A,B) :- directed(A,B)",8,0.301040,0.101806,0.104007,0.347231,0.040352,0.301040,0.166163,0.166163,0.030104,0.166163
1,actedin,0.3,0.2,"actedin(A,B) :- directed(A,B)",8,0.150520,0.101806,0.104007,0.347231,0.040352,0.150520,0.166163,0.166163,0.030104,0.166163
2,actedin,0.3,0.3,"actedin(A,B) :- directed(A,B)",8,0.100347,0.101806,0.104007,0.347231,0.040352,0.100347,0.166163,0.166163,0.030104,0.166163
3,actedin,0.3,0.4,"actedin(A,B) :- directed(A,B)",8,0.075260,0.101806,0.104007,0.347231,0.040352,0.075260,0.166163,0.166163,0.030104,0.166163
4,actedin,0.3,0.5,"actedin(A,B) :- directed(A,B)",8,0.060208,0.101806,0.104007,0.347231,0.040352,0.060208,0.166163,0.166163,0.030104,0.166163


## 2. Only keep a subset of rules
### 2.1. Only keep the non-recursive rules; drop recursive rules

In [7]:
from kbc_pul.data_structures.rule_wrapper import get_pylo_rule_from_string, is_pylo_rule_recursive


def is_rule_recursive(rule_string: str) -> bool:
        pylo_rule: PyloClause = get_pylo_rule_from_string(rule_string)
        is_rule_recursive = is_pylo_rule_recursive(pylo_rule)
        return is_rule_recursive


mask_recursive_rules = df_rule_wrappers_all_targets.apply(
    lambda row: is_rule_recursive(row["Rule"]),
    axis=1
)

In [8]:
print(len(df_rule_wrappers_all_targets))
df_rule_wrappers_all_targets: pd.DataFrame = df_rule_wrappers_all_targets[~mask_recursive_rules]
print(len(df_rule_wrappers_all_targets))



29550
9590


### 2.3 Drop the Pair-positive columns (both directions)

In [9]:
df_rule_wrappers_all_targets.drop(
        [ConfidenceEnum.TRUE_CONF_BIAS_YS_ZERO_S_TO_O.value,
         ConfidenceEnum.TRUE_CONF_BIAS_YS_ZERO_O_TO_S.value],
        axis=1,
        inplace=True,
        errors='ignore'
)
df_rule_wrappers_all_targets.head()

,target_relation,true_label_frequency,noisy_label_frequency,Rule,random_trial_index,ICW,$conf$,PCA ${p^{-1}}$,IPW,IPW-PCA $p$,IPW-PCA ${p^{-1}}$,CWA,PCA $p$
0,actedin,0.3,0.1,"actedin(A,B) :- directed(A,B)",8,0.301040,0.101806,0.040352,0.301040,0.166163,0.166163,0.030104,0.166163
1,actedin,0.3,0.2,"actedin(A,B) :- directed(A,B)",8,0.150520,0.101806,0.040352,0.150520,0.166163,0.166163,0.030104,0.166163
2,actedin,0.3,0.3,"actedin(A,B) :- directed(A,B)",8,0.100347,0.101806,0.040352,0.100347,0.166163,0.166163,0.030104,0.166163
3,actedin,0.3,0.4,"actedin(A,B) :- directed(A,B)",8,0.075260,0.101806,0.040352,0.075260,0.166163,0.166163,0.030104,0.166163
4,actedin,0.3,0.5,"actedin(A,B) :- directed(A,B)",8,0.060208,0.101806,0.040352,0.060208,0.166163,0.166163,0.030104,0.166163


### 2.4 Drop the IPW-PCA columns (both directions)

In [10]:
df_rule_wrappers_all_targets.drop(
        [ConfidenceEnum.IPW_PCA_CONF_S_TO_O.value,
         ConfidenceEnum.IPW_PCA_CONF_O_TO_S.value],
        axis=1,
        inplace=True,
        errors='ignore'
)
df_rule_wrappers_all_targets.head()

,target_relation,true_label_frequency,noisy_label_frequency,Rule,random_trial_index,ICW,$conf$,PCA ${p^{-1}}$,IPW,CWA,PCA $p$
0,actedin,0.3,0.1,"actedin(A,B) :- directed(A,B)",8,0.301040,0.101806,0.040352,0.301040,0.030104,0.166163
1,actedin,0.3,0.2,"actedin(A,B) :- directed(A,B)",8,0.150520,0.101806,0.040352,0.150520,0.030104,0.166163
2,actedin,0.3,0.3,"actedin(A,B) :- directed(A,B)",8,0.100347,0.101806,0.040352,0.100347,0.030104,0.166163
3,actedin,0.3,0.4,"actedin(A,B) :- directed(A,B)",8,0.075260,0.101806,0.040352,0.075260,0.030104,0.166163
4,actedin,0.3,0.5,"actedin(A,B) :- directed(A,B)",8,0.060208,0.101806,0.040352,0.060208,0.030104,0.166163


### 2.4 Drop the ICW column

In [11]:
df_rule_wrappers_all_targets.drop(
        [ConfidenceEnum.ICW_CONF.value],
        axis=1,
        inplace=True,
        errors='ignore'
)
df_rule_wrappers_all_targets.head()


,target_relation,true_label_frequency,noisy_label_frequency,Rule,random_trial_index,$conf$,PCA ${p^{-1}}$,IPW,CWA,PCA $p$
0,actedin,0.3,0.1,"actedin(A,B) :- directed(A,B)",8,0.101806,0.040352,0.301040,0.030104,0.166163
1,actedin,0.3,0.2,"actedin(A,B) :- directed(A,B)",8,0.101806,0.040352,0.150520,0.030104,0.166163
2,actedin,0.3,0.3,"actedin(A,B) :- directed(A,B)",8,0.101806,0.040352,0.100347,0.030104,0.166163
3,actedin,0.3,0.4,"actedin(A,B) :- directed(A,B)",8,0.101806,0.040352,0.075260,0.030104,0.166163
4,actedin,0.3,0.5,"actedin(A,B) :- directed(A,B)",8,0.101806,0.040352,0.060208,0.030104,0.166163


### 2.4 Only use rules with 10 trials

In [12]:
df_count_trials: pd.DataFrame = df_rule_wrappers_all_targets[
      [
        "target_relation",
        'true_label_frequency',
        'noisy_label_frequency',
        "Rule",
        "random_trial_index"
    ]
].groupby(
    [
        "target_relation",
        'true_label_frequency',
        'noisy_label_frequency',
        "Rule",
    ]
).count().reset_index()

In [13]:
df_less_than_ten_trials: pd.DataFrame = df_count_trials[df_count_trials["random_trial_index"].values != 10]
df_less_than_ten_trials

,target_relation,true_label_frequency,noisy_label_frequency,Rule,random_trial_index
540,isaffiliatedto,0.3,0.1,"isaffiliatedto(A,B) :- playsfor(A,B)",9
541,isaffiliatedto,0.3,0.2,"isaffiliatedto(A,B) :- playsfor(A,B)",9
542,isaffiliatedto,0.3,0.3,"isaffiliatedto(A,B) :- playsfor(A,B)",9
543,isaffiliatedto,0.3,0.4,"isaffiliatedto(A,B) :- playsfor(A,B)",9
544,isaffiliatedto,0.3,0.5,"isaffiliatedto(A,B) :- playsfor(A,B)",9
545,isaffiliatedto,0.3,0.6,"isaffiliatedto(A,B) :- playsfor(A,B)",9
546,isaffiliatedto,0.3,0.7,"isaffiliatedto(A,B) :- playsfor(A,B)",9
547,isaffiliatedto,0.3,0.8,"isaffiliatedto(A,B) :- playsfor(A,B)",9
548,isaffiliatedto,0.3,0.9,"isaffiliatedto(A,B) :- playsfor(A,B)",9
549,isaffiliatedto,0.3,1.0,"isaffiliatedto(A,B) :- playsfor(A,B)",9


In [14]:
df_less_than_ten_trials_filter_info = df_less_than_ten_trials[["target_relation", "Rule"]].drop_duplicates()
df_less_than_ten_trials_filter_info.head()

,target_relation,Rule
540,isaffiliatedto,"isaffiliatedto(A,B) :- playsfor(A,B)"
900,playsfor,"playsfor(A,B) :- isaffiliatedto(A,B)"


In [15]:
for row_index, row in df_less_than_ten_trials_filter_info.iterrows():
    target_relation = row["target_relation"]
    rule = row["Rule"]
    df_rule_wrappers_all_targets = df_rule_wrappers_all_targets[
        ~(
        (df_rule_wrappers_all_targets["target_relation"] == target_relation)
        &
        (df_rule_wrappers_all_targets["Rule"]==rule)
        )
    ]
df_rule_wrappers_all_targets.head()

,target_relation,true_label_frequency,noisy_label_frequency,Rule,random_trial_index,$conf$,PCA ${p^{-1}}$,IPW,CWA,PCA $p$
0,actedin,0.3,0.1,"actedin(A,B) :- directed(A,B)",8,0.101806,0.040352,0.301040,0.030104,0.166163
1,actedin,0.3,0.2,"actedin(A,B) :- directed(A,B)",8,0.101806,0.040352,0.150520,0.030104,0.166163
2,actedin,0.3,0.3,"actedin(A,B) :- directed(A,B)",8,0.101806,0.040352,0.100347,0.030104,0.166163
3,actedin,0.3,0.4,"actedin(A,B) :- directed(A,B)",8,0.101806,0.040352,0.075260,0.030104,0.166163
4,actedin,0.3,0.5,"actedin(A,B) :- directed(A,B)",8,0.101806,0.040352,0.060208,0.030104,0.166163


**Now, we have the full dataframe**

****

## Calculate $[conf(R) - \widehat{conf}(R)]$

In [16]:
true_conf: ConfidenceEnum = ConfidenceEnum.TRUE_CONF

conf_estimators_list: List[ConfidenceEnum] = [
    ConfidenceEnum.CWA_CONF,
    # ConfidenceEnum.ICW_CONF,
    ConfidenceEnum.PCA_CONF_S_TO_O,
    ConfidenceEnum.PCA_CONF_O_TO_S,
     ConfidenceEnum.IPW_CONF,
]
all_confs_list: List[ConfidenceEnum] = [ConfidenceEnum.TRUE_CONF ] + conf_estimators_list

column_names_all_confs: List[str] = [
    conf.get_name()
    for conf in all_confs_list
]

In [17]:
df_rule_wrappers_all_targets = df_rule_wrappers_all_targets[
    column_names_logistics + column_names_all_confs
]
df_rule_wrappers_all_targets.head()

,target_relation,true_label_frequency,noisy_label_frequency,Rule,random_trial_index,$conf$,CWA,PCA $p$,PCA ${p^{-1}}$,IPW
0,actedin,0.3,0.1,"actedin(A,B) :- directed(A,B)",8,0.101806,0.030104,0.166163,0.040352,0.301040
1,actedin,0.3,0.2,"actedin(A,B) :- directed(A,B)",8,0.101806,0.030104,0.166163,0.040352,0.150520
2,actedin,0.3,0.3,"actedin(A,B) :- directed(A,B)",8,0.101806,0.030104,0.166163,0.040352,0.100347
3,actedin,0.3,0.4,"actedin(A,B) :- directed(A,B)",8,0.101806,0.030104,0.166163,0.040352,0.075260
4,actedin,0.3,0.5,"actedin(A,B) :- directed(A,B)",8,0.101806,0.030104,0.166163,0.040352,0.060208


In [18]:
df_conf_estimators_true_other = df_rule_wrappers_all_targets[
    df_rule_wrappers_all_targets["true_label_frequency"]
    == df_rule_wrappers_all_targets["noisy_label_frequency"]
]
df_conf_estimators_true_other.head()

,target_relation,true_label_frequency,noisy_label_frequency,Rule,random_trial_index,$conf$,CWA,PCA $p$,PCA ${p^{-1}}$,IPW
2,actedin,0.3,0.3,"actedin(A,B) :- directed(A,B)",8,0.101806,0.030104,0.166163,0.040352,0.100347
12,actedin,0.3,0.3,"actedin(A,B) :- directed(A,B)",9,0.101806,0.032841,0.172745,0.044042,0.109469
22,actedin,0.3,0.3,"actedin(A,B) :- directed(A,B)",4,0.101806,0.031928,0.172925,0.042966,0.106428
32,actedin,0.3,0.3,"actedin(A,B) :- directed(A,B)",6,0.101806,0.029557,0.164969,0.039892,0.098522
42,actedin,0.3,0.3,"actedin(A,B) :- directed(A,B)",1,0.101806,0.030834,0.167161,0.041852,0.102779


In [19]:
column_names_info = ColumnNamesInfo(
        true_conf=true_conf,
        column_name_true_conf=true_conf.get_name(),
        conf_estimators=conf_estimators_list,
        column_names_conf_estimators=[
            col.get_name()
            for col in conf_estimators_list
        ],
        column_names_logistics=column_names_logistics
    )


In [20]:
def get_df_rulewise_squared_diffs_between_true_conf_and_conf_estimator(
        df_rule_wrappers: pd.DataFrame,
        column_names_info: ColumnNamesInfo
) -> pd.DataFrame:
    df_rulewise_diffs_between_true_conf_and_conf_estimator: pd.DataFrame = df_rule_wrappers[
        column_names_info.column_names_logistics
    ]

    col_name_estimator: str
    for col_name_estimator in column_names_info.column_names_conf_estimators:
        df_rulewise_diffs_between_true_conf_and_conf_estimator \
            = df_rulewise_diffs_between_true_conf_and_conf_estimator.assign(
                **{
                    col_name_estimator: (
                        (df_rule_wrappers[column_names_info.column_name_true_conf]
                            - df_rule_wrappers[col_name_estimator]) ** 2
                    )
                }
            )
    return df_rulewise_diffs_between_true_conf_and_conf_estimator

df_conf_squared_errors: pd.DataFrame = get_df_rulewise_squared_diffs_between_true_conf_and_conf_estimator(
    df_rule_wrappers=df_rule_wrappers_all_targets,
    column_names_info = column_names_info
)
df_conf_squared_errors.head()

,target_relation,true_label_frequency,noisy_label_frequency,Rule,random_trial_index,CWA,PCA $p$,PCA ${p^{-1}}$,IPW
0,actedin,0.3,0.1,"actedin(A,B) :- directed(A,B)",8,0.005141,0.004142,0.003777,0.039694
1,actedin,0.3,0.2,"actedin(A,B) :- directed(A,B)",8,0.005141,0.004142,0.003777,0.002373
2,actedin,0.3,0.3,"actedin(A,B) :- directed(A,B)",8,0.005141,0.004142,0.003777,0.000002
3,actedin,0.3,0.4,"actedin(A,B) :- directed(A,B)",8,0.005141,0.004142,0.003777,0.000705
4,actedin,0.3,0.5,"actedin(A,B) :- directed(A,B)",8,0.005141,0.004142,0.003777,0.001730


## AVERAGE the PCA(S) and PCA(O)

In [21]:
df_conf_squared_errors.columns

Index(['target_relation', 'true_label_frequency', 'noisy_label_frequency',
       'Rule', 'random_trial_index', 'CWA', 'PCA $p$', 'PCA ${p^{-1}}$',
       'IPW'],
      dtype='object')

In [22]:
df_conf_squared_errors["PCA"] = (
    (
            df_conf_squared_errors[ConfidenceEnum.PCA_CONF_S_TO_O.value]
            +
            df_conf_squared_errors[ConfidenceEnum.PCA_CONF_O_TO_S.value]
    ) / 2
)
df_conf_squared_errors.head()

,target_relation,true_label_frequency,noisy_label_frequency,Rule,random_trial_index,CWA,PCA $p$,PCA ${p^{-1}}$,IPW,PCA
0,actedin,0.3,0.1,"actedin(A,B) :- directed(A,B)",8,0.005141,0.004142,0.003777,0.039694,0.003959
1,actedin,0.3,0.2,"actedin(A,B) :- directed(A,B)",8,0.005141,0.004142,0.003777,0.002373,0.003959
2,actedin,0.3,0.3,"actedin(A,B) :- directed(A,B)",8,0.005141,0.004142,0.003777,0.000002,0.003959
3,actedin,0.3,0.4,"actedin(A,B) :- directed(A,B)",8,0.005141,0.004142,0.003777,0.000705,0.003959
4,actedin,0.3,0.5,"actedin(A,B) :- directed(A,B)",8,0.005141,0.004142,0.003777,0.001730,0.003959


In [23]:
df_conf_squared_errors = df_conf_squared_errors.drop(
        columns=[
            ConfidenceEnum.PCA_CONF_S_TO_O.value,
            ConfidenceEnum.PCA_CONF_O_TO_S.value
        ],
        axis=1,
        errors='ignore'
    )
df_conf_squared_errors.head()

,target_relation,true_label_frequency,noisy_label_frequency,Rule,random_trial_index,CWA,IPW,PCA
0,actedin,0.3,0.1,"actedin(A,B) :- directed(A,B)",8,0.005141,0.039694,0.003959
1,actedin,0.3,0.2,"actedin(A,B) :- directed(A,B)",8,0.005141,0.002373,0.003959
2,actedin,0.3,0.3,"actedin(A,B) :- directed(A,B)",8,0.005141,0.000002,0.003959
3,actedin,0.3,0.4,"actedin(A,B) :- directed(A,B)",8,0.005141,0.000705,0.003959
4,actedin,0.3,0.5,"actedin(A,B) :- directed(A,B)",8,0.005141,0.001730,0.003959


# Now start averaging

In [24]:
df_conf_squared_errors_avg_over_trials: pd.DataFrame = df_conf_squared_errors.groupby(
    by=["target_relation", 'true_label_frequency', "noisy_label_frequency", "Rule"],
    sort=True,
    as_index=False
).mean()

df_conf_squared_errors_avg_over_trials.head()

,target_relation,true_label_frequency,noisy_label_frequency,Rule,random_trial_index,CWA,IPW,PCA
0,actedin,0.3,0.1,"actedin(A,B) :- directed(A,B)",4.5,0.005044,0.042676,0.003983
1,actedin,0.3,0.2,"actedin(A,B) :- directed(A,B)",4.5,0.005044,0.002766,0.003983
2,actedin,0.3,0.3,"actedin(A,B) :- directed(A,B)",4.5,0.005044,0.000020,0.003983
3,actedin,0.3,0.4,"actedin(A,B) :- directed(A,B)",4.5,0.005044,0.000626,0.003983
4,actedin,0.3,0.5,"actedin(A,B) :- directed(A,B)",4.5,0.005044,0.001624,0.003983


In [25]:
df_conf_squared_errors_avg_over_trials_and_rules: pd.DataFrame = df_conf_squared_errors_avg_over_trials.groupby(
    by=["target_relation", 'true_label_frequency', "noisy_label_frequency",],
    sort=True,
    as_index=False
).mean()
df_conf_squared_errors_avg_over_trials_and_rules.head()

,target_relation,true_label_frequency,noisy_label_frequency,random_trial_index,CWA,IPW,PCA
0,actedin,0.3,0.1,4.5,0.005044,0.042676,0.003983
1,actedin,0.3,0.2,4.5,0.005044,0.002766,0.003983
2,actedin,0.3,0.3,4.5,0.005044,0.000020,0.003983
3,actedin,0.3,0.4,4.5,0.005044,0.000626,0.003983
4,actedin,0.3,0.5,4.5,0.005044,0.001624,0.003983


In [26]:
len(df_conf_squared_errors_avg_over_trials_and_rules)

420

## Subset of noisy_other

In [27]:
first_true_label_freq_to_include = 0.3
second_true_label_freq_to_include = 0.7

true_label_frequencies_set: Set[float] = {
    first_true_label_freq_to_include, second_true_label_freq_to_include,
}
true_label_frequency_to_estimate_map: Dict[float, Set[float]] = dict()

label_frequency_est_diff: float = 0.1
label_frequencies_to_keep: Set[float] = set(true_label_frequencies_set)
for true_label_freq in true_label_frequencies_set:
    true_label_frequency_to_estimate_map[true_label_freq] = {
        round(true_label_freq - label_frequency_est_diff, 1),
        round(true_label_freq + label_frequency_est_diff, 1)
    }
    label_frequencies_to_keep.update(true_label_frequency_to_estimate_map[true_label_freq])

In [28]:
df_conf_errors_avg_over_trials_and_rules_subset = df_conf_squared_errors_avg_over_trials_and_rules[
    df_conf_squared_errors_avg_over_trials_and_rules["noisy_label_frequency"].isin(label_frequencies_to_keep)
]
df_conf_errors_avg_over_trials_and_rules_subset.head()

,target_relation,true_label_frequency,noisy_label_frequency,random_trial_index,CWA,IPW,PCA
1,actedin,0.3,0.2,4.5,0.005044,0.002766,0.003983
2,actedin,0.3,0.3,4.5,0.005044,0.000020,0.003983
3,actedin,0.3,0.4,4.5,0.005044,0.000626,0.003983
5,actedin,0.3,0.6,4.5,0.005044,0.002553,0.003983
6,actedin,0.3,0.7,4.5,0.005044,0.003345,0.003983


In [29]:
len(df_conf_errors_avg_over_trials_and_rules_subset)

252

## Count the rules per $p$

In [30]:
df_n_rules_per_target = df_rule_wrappers_all_targets[["target_relation", "Rule"]].groupby(
    by=['target_relation'],
    # sort=True,
    # as_index=False
)["Rule"].nunique().to_frame().reset_index().rename(
    columns={"Rule" : "# rules"}
)


df_n_rules_per_target.head()




,target_relation,# rules
0,actedin,1
1,created,2
2,dealswith,7
3,diedin,1
4,directed,2


****
# Format pretty table

Goal:
* put smallest value per row in BOLT
* per target: mean_value 0.3 / 0.4

In [31]:
true_label_freq_to_noisy_to_df_map: Dict[float, Dict[float, pd.DataFrame]] = dict()
for true_label_freq in true_label_frequencies_set:
    df_true_tmp: pd.DataFrame = df_conf_errors_avg_over_trials_and_rules_subset[
        df_conf_errors_avg_over_trials_and_rules_subset["true_label_frequency"] == true_label_freq
    ]
    noisy_label_freq_to_df_map = dict()
    true_label_freq_to_noisy_to_df_map[true_label_freq] = noisy_label_freq_to_df_map

    df_true_and_noisy_tmp = df_true_tmp[
        df_true_tmp["noisy_label_frequency"] == true_label_freq
    ]
    noisy_label_freq_to_df_map[true_label_freq] = df_true_and_noisy_tmp[
        [col for col in df_true_and_noisy_tmp.columns if col != "noisy_label_frequency" and col != "true_label_frequency"]
    ]

    for noisy_label_freq in true_label_frequency_to_estimate_map[true_label_freq]:
        df_true_and_noisy_tmp = df_true_tmp[
            df_true_tmp["noisy_label_frequency"] == noisy_label_freq
        ]
        noisy_label_freq_to_df_map[noisy_label_freq] = df_true_and_noisy_tmp[
            [col for col in df_true_and_noisy_tmp.columns if col != "noisy_label_frequency" and col != "true_label_frequency"]
        ]

true_label_freq_to_noisy_to_df_map[first_true_label_freq_to_include][0.2].head()

,target_relation,random_trial_index,CWA,IPW,PCA
1,actedin,4.5,0.005044,0.002766,0.003983
21,created,4.5,0.047558,0.023654,0.004257
41,dealswith,4.5,0.017872,0.010924,0.009553
61,diedin,4.5,0.008779,0.003866,0.010935
81,directed,4.5,0.056130,0.034786,0.006112


In [32]:
from typing import Iterator

true_label_freq_to_df_map = dict()

label_freq_estimators: Iterator[float]
for true_label_freq in true_label_frequencies_set:
    noisy_to_df_map: Dict[float, pd.DataFrame]  = true_label_freq_to_noisy_to_df_map[true_label_freq]

    df_true_label_freq: pd.DataFrame = noisy_to_df_map[true_label_freq]

    lower_est: float = round(true_label_freq - label_frequency_est_diff, 1)
    higher_est: float = round(true_label_freq + label_frequency_est_diff, 1)


    df_lower: pd.DataFrame = noisy_to_df_map[lower_est][
        ['target_relation', ConfidenceEnum.IPW_CONF.value]
    ].rename(
        columns={
            ConfidenceEnum.IPW_CONF.value: f"{ConfidenceEnum.IPW_CONF.value}_lower"
        }
    )

    df_true_label_freq = pd.merge(
        left=df_true_label_freq,
        right=df_lower,
        on="target_relation"
    )

    df_higher = noisy_to_df_map[higher_est][
        ['target_relation', ConfidenceEnum.IPW_CONF.value]
    ].rename(
        columns={
            ConfidenceEnum.IPW_CONF.value: f"{ConfidenceEnum.IPW_CONF.value}_higher"
        }
    )
    df_true_label_freq = pd.merge(
        left=df_true_label_freq,
        right=df_higher,
        on="target_relation"
    )
    true_label_freq_to_df_map[true_label_freq] = df_true_label_freq

true_label_freq_to_df_map[0.3].head()

,target_relation,random_trial_index,CWA,IPW,PCA,IPW_lower,IPW_higher
0,actedin,4.5,0.005044,0.000020,0.003983,0.002766,0.000626
1,created,4.5,0.047558,0.000413,0.004257,0.023654,0.006579
2,dealswith,4.5,0.017872,0.000274,0.009553,0.010924,0.002205
3,diedin,4.5,0.008779,0.000057,0.010935,0.003866,0.001290
4,directed,4.5,0.056130,0.000674,0.006112,0.034786,0.006657


In [33]:
for key, df in true_label_freq_to_df_map.items():
    true_label_freq_to_df_map[key] = df.drop(
        columns=["random_trial_index"],
        axis=1,
        errors='ignore'
    )


In [34]:
df_one_row_per_target = pd.merge(
    left=true_label_freq_to_df_map[first_true_label_freq_to_include],
    right=true_label_freq_to_df_map[second_true_label_freq_to_include],
    on="target_relation",
    suffixes=(f"_{first_true_label_freq_to_include}", f"_{second_true_label_freq_to_include}")
)
df_one_row_per_target.head()

,target_relation,CWA_0.3,IPW_0.3,PCA_0.3,IPW_lower_0.3,IPW_higher_0.3,CWA_0.7,IPW_0.7,PCA_0.7,IPW_lower_0.7,IPW_higher_0.7
0,actedin,0.005044,0.000020,0.003983,0.002766,0.000626,0.000868,0.000007,0.014121,0.000360,0.000132
1,created,0.047558,0.000413,0.004257,0.023654,0.006579,0.008514,0.000157,0.011084,0.003092,0.001508
2,dealswith,0.017872,0.000274,0.009553,0.010924,0.002205,0.003405,0.000046,0.000714,0.001045,0.000640
3,diedin,0.008779,0.000057,0.010935,0.003866,0.001290,0.001573,0.000007,0.008410,0.000504,0.000275
4,directed,0.056130,0.000674,0.006112,0.034786,0.006657,0.010634,0.000058,0.028192,0.003232,0.001920


## What is the smallest value?

In [35]:
all_values: np.ndarray = df_one_row_per_target[
    [ col
      for col in df_one_row_per_target.columns
      if col != "target_relation"
      ]
].values

min_val = np.amin(all_values)
min_val

5.206077916404551e-06

In [36]:
min_val * 10000

0.052060779164045515

In [37]:
max_val = np.amax(all_values)
max_val

0.1517314056437167

In [38]:
max_val * 10000

1517.314056437167

In [39]:
df_one_row_per_target.head() * 10000

,target_relation,CWA_0.3,IPW_0.3,PCA_0.3,IPW_lower_0.3,IPW_higher_0.3,CWA_0.7,IPW_0.7,PCA_0.7,IPW_lower_0.7,IPW_higher_0.7
0,actedinactedinactedinactedinactedinactedinacte...,50.439792,0.197210,39.831514,27.655205,6.263698,8.682130,0.069170,141.205257,3.602644,1.318339
1,createdcreatedcreatedcreatedcreatedcreatedcrea...,475.575363,4.133103,42.567653,236.538050,65.789823,85.142179,1.565609,110.842459,30.916689,15.075349
2,dealswithdealswithdealswithdealswithdealswithd...,178.717080,2.741393,95.530126,109.236689,22.053812,34.049894,0.463346,7.141257,10.450022,6.401090
3,diedindiedindiedindiedindiedindiedindiedindied...,87.793571,0.574204,109.346092,38.655024,12.902659,15.731572,0.065602,84.097181,5.037616,2.746903
4,directeddirecteddirecteddirecteddirecteddirect...,561.303077,6.743527,61.118245,347.856904,66.570383,106.341465,0.582379,281.916339,32.324272,19.196401


In [40]:
df_one_row_per_target.dtypes

target_relation     object
CWA_0.3            float64
IPW_0.3            float64
PCA_0.3            float64
IPW_lower_0.3      float64
IPW_higher_0.3     float64
CWA_0.7            float64
IPW_0.7            float64
PCA_0.7            float64
IPW_lower_0.7      float64
IPW_higher_0.7     float64
dtype: object

In [41]:
exponent = 4

multiplication_factor = 10 ** exponent
multiplication_factor

10000

In [42]:
df_one_row_per_target[
    df_one_row_per_target.select_dtypes(include=['number']).columns
] *= multiplication_factor

df_one_row_per_target

,target_relation,CWA_0.3,IPW_0.3,PCA_0.3,IPW_lower_0.3,IPW_higher_0.3,CWA_0.7,IPW_0.7,PCA_0.7,IPW_lower_0.7,IPW_higher_0.7
0,actedin,50.439792,0.197210,39.831514,27.655205,6.263698,8.682130,0.069170,141.205257,3.602644,1.318339
1,created,475.575363,4.133103,42.567653,236.538050,65.789823,85.142179,1.565609,110.842459,30.916689,15.075349
2,dealswith,178.717080,2.741393,95.530126,109.236689,22.053812,34.049894,0.463346,7.141257,10.450022,6.401090
3,diedin,87.793571,0.574204,109.346092,38.655024,12.902659,15.731572,0.065602,84.097181,5.037616,2.746903
4,directed,561.303077,6.743527,61.118245,347.856904,66.570383,106.341465,0.582379,281.916339,32.324272,19.196401
5,exports,115.306671,6.114928,67.984786,70.087861,18.493160,22.447539,1.572410,9.904105,7.967391,5.210053
6,graduatedfrom,255.439036,3.019419,128.908457,147.570466,32.259762,46.070790,0.556970,17.419621,16.802020,7.840607
7,happenedin,406.930870,0.749552,267.301350,204.799698,53.155623,75.968555,0.064481,22.258161,21.786701,13.721793
8,hascapital,147.471250,8.152137,667.913283,64.349069,28.713229,28.295549,1.237589,796.841151,7.716939,6.461616
9,hasneighbor,232.432138,11.215188,84.933150,170.938947,30.645766,38.980353,4.452073,4.032811,25.998695,7.383825


## Output files definitions

In [43]:
df_one_row_per_target.head()

,target_relation,CWA_0.3,IPW_0.3,PCA_0.3,IPW_lower_0.3,IPW_higher_0.3,CWA_0.7,IPW_0.7,PCA_0.7,IPW_lower_0.7,IPW_higher_0.7
0,actedin,50.439792,0.197210,39.831514,27.655205,6.263698,8.682130,0.069170,141.205257,3.602644,1.318339
1,created,475.575363,4.133103,42.567653,236.538050,65.789823,85.142179,1.565609,110.842459,30.916689,15.075349
2,dealswith,178.717080,2.741393,95.530126,109.236689,22.053812,34.049894,0.463346,7.141257,10.450022,6.401090
3,diedin,87.793571,0.574204,109.346092,38.655024,12.902659,15.731572,0.065602,84.097181,5.037616,2.746903
4,directed,561.303077,6.743527,61.118245,347.856904,66.570383,106.341465,0.582379,281.916339,32.324272,19.196401


In [56]:
dir_latex_table: str = os.path.join(
    kbc_e_metrics_project_dir,
    "paper_latex_tables",
    'known_prop_scores',
    'scar'
)

if not os.path.exists(dir_latex_table):
    os.makedirs(dir_latex_table)

filename_tsv_rule_stats = os.path.join(
    dir_latex_table,
    "conf_error_stats_v4.tsv"
)
filename_tsv_single_row_summary = os.path.join(
    dir_latex_table,
    "noisy_scar_single_row_summary.tsv"
)
print(filename_tsv_single_row_summary)

/home/joschout/Documents/Repos/KUL-PUL/paper_latex_tables/known_prop_scores/scar/noisy_scar_single_row_summary.tsv


## Create single-row summary

In [45]:
df_one_row_in_total: pd.Series = df_one_row_per_target.mean(
)


df_one_row_in_total

/tmp/ipykernel_54658/1788372146.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_one_row_in_total: pd.Series = df_one_row_per_target.mean(


CWA_0.3           292.534231
IPW_0.3             4.641863
PCA_0.3           192.732178
IPW_lower_0.3     154.171506
IPW_higher_0.3     40.866744
CWA_0.7            53.780402
IPW_0.7             1.088689
PCA_0.7           173.296008
IPW_lower_0.7      18.123566
IPW_higher_0.7     10.019430
dtype: float64

In [46]:
df_n_rules_per_target.head()

,target_relation,# rules
0,actedin,1
1,created,2
2,dealswith,7
3,diedin,1
4,directed,2


In [47]:
df_one_row_in_total["# rules"] = int(df_n_rules_per_target["# rules"].sum())
df_one_row_in_total

CWA_0.3           292.534231
IPW_0.3             4.641863
PCA_0.3           192.732178
IPW_lower_0.3     154.171506
IPW_higher_0.3     40.866744
CWA_0.7            53.780402
IPW_0.7             1.088689
PCA_0.7           173.296008
IPW_lower_0.7      18.123566
IPW_higher_0.7     10.019430
# rules            47.000000
dtype: float64

In [48]:
type(df_one_row_in_total)

pandas.core.series.Series

In [49]:
df_one_row_in_total.to_csv(
    filename_tsv_single_row_summary,
    sep = "\t",
    header=None
)


### Now create a pretty table

In [50]:
column_names_info.column_names_conf_estimators

['CWA', 'PCA $p$', 'PCA ${p^{-1}}$', 'IPW']

In [51]:
simplified_column_names_conf_estimators = ['CWA', 'PCA', 'IPW',]

In [52]:
multi_index_columns = [
    ("$p$", ""),
    ("\# R", "")
]
from itertools import product

# conf_upper_cols = column_names_info.column_names_conf_estimators + [
#     f"{ConfidenceEnum.IPW_CONF.value} " + "($\Delta c=-" + f"{label_frequency_est_diff}" + "$)",
#     f"{ConfidenceEnum.IPW_CONF.value} " + "($\Delta c=" + f"{label_frequency_est_diff}" + "$)",
# ]
conf_upper_cols  = simplified_column_names_conf_estimators + [
    f"{ConfidenceEnum.IPW_CONF.value} " + "($-\Delta$)",
    f"{ConfidenceEnum.IPW_CONF.value} " + "($+\Delta$)",
]

c_subcols = ["$c=0.3$", "$c=0.7$"]

multi_index_columns = multi_index_columns + list(product(c_subcols, conf_upper_cols))
# multi_index_list

multi_index_columns = pd.MultiIndex.from_tuples(multi_index_columns)
multi_index_columns

MultiIndex([(    '$p$',                ''),
            (   '\# R',                ''),
            ('$c=0.3$',             'CWA'),
            ('$c=0.3$',             'PCA'),
            ('$c=0.3$',             'IPW'),
            ('$c=0.3$', 'IPW ($-\Delta$)'),
            ('$c=0.3$', 'IPW ($+\Delta$)'),
            ('$c=0.7$',             'CWA'),
            ('$c=0.7$',             'PCA'),
            ('$c=0.7$',             'IPW'),
            ('$c=0.7$', 'IPW ($-\Delta$)'),
            ('$c=0.7$', 'IPW ($+\Delta$)')],
           )

In [53]:

rule_counter: int = 1
rule_str_to_rule_id_map: Dict[str, int] = {}


float_precision: int = 1

col_name_conf_estimator: str


pretty_rows: List[List] = []

row_index: int
row: pd.Series

# columns_to_use = [
#     "$p$",
#     "\# rules"
# ] + column_names_info.column_names_conf_estimators + [
#     f"{ConfidenceEnum.IPW_CONF.value} " + "($\Delta c=-" + f"{label_frequency_est_diff}" + "$)",
#     f"{ConfidenceEnum.IPW_CONF.value} " + "($\Delta c=" + f"{label_frequency_est_diff}" + "$)",
# ]

LabelFreq = float
def get_dict_with_smallest_estimator_per_label_freq(row: pd.Series) -> Dict[LabelFreq, Set[str]]:
    # Find estimator with smallest mean value for label frequency###################
    label_freq_to_set_of_smallest_est_map: Dict[LabelFreq, Set[str]] = dict()
    for label_freq in [first_true_label_freq_to_include, second_true_label_freq_to_include]:
        o_set_of_col_names_with_min_value: Optional[Set[str]] = None
        o_current_smallest_value: Optional[float] = None
        # Find smallest squared error
        for col_name_conf_estimator in simplified_column_names_conf_estimators:
            current_val: float = row[f"{col_name_conf_estimator}_{label_freq}"]
            # print(current_val)
            if o_set_of_col_names_with_min_value is None or o_current_smallest_value > current_val:
                o_set_of_col_names_with_min_value = {col_name_conf_estimator}
                o_current_smallest_value = current_val
            elif current_val == o_current_smallest_value:
                o_set_of_col_names_with_min_value.update(col_name_conf_estimator)

        label_freq_to_set_of_smallest_est_map[label_freq] = o_set_of_col_names_with_min_value
    return label_freq_to_set_of_smallest_est_map


def format_value_depending_on_whether_it_is_smallest(
        value: float,
        is_smallest: bool,
        float_precision: float,
        use_si: bool = False
)-> str:
    if is_smallest:
        if not use_si:
            formatted_value = "$\\bm{" + f"{value:0.{float_precision}f}" + "}$"
            # formatted_value = "$\\bm{" + f"{value:0.{float_precision}e}" + "}$"
        else:
            formatted_value = "\\textbf{$" + f"\\num[round-precision={float_precision},round-mode=figures,scientific-notation=true]"+\
                              "{"+ str(value) + "}"+ "$}"
    else:
        if not use_si:
            formatted_value = f"${value:0.{float_precision}f}$"
            # formatted_value = f"${value:0.{float_precision}e}$"
        else:
            formatted_value = "$" + f"\\num[round-precision={float_precision},round-mode=figures,scientific-notation=true]"+\
                              "{"+ str(value) + "}"+ "$"

    return formatted_value


estimator_columns = simplified_column_names_conf_estimators + [
    f"{ConfidenceEnum.IPW_CONF.value}_lower",
    f"{ConfidenceEnum.IPW_CONF.value}_higher"
]

# For each row, i.e. for each target relation
for row_index, row in df_one_row_per_target.iterrows():

    # Find estimator with smallest mean value for label frequency###################
    label_freq_to_set_of_smallest_est_map: Dict[float, Set[str]] = get_dict_with_smallest_estimator_per_label_freq(
        row=row
    )
    ##################################################################################
    # Construct the new row
    ######################

    target_relation = row["target_relation"]


    nb_of_rules = df_n_rules_per_target[df_n_rules_per_target['target_relation'] == target_relation][
        "# rules"
    ].iloc[0]

    new_row: List[str] = [
        target_relation,
        nb_of_rules
    ]
    # For each Confidence estimator, get the value at c 0.3 and 0.7
    # for col_name_conf_estimator in estimator_columns:
    #     mean_val_03:float = row[f"{col_name_conf_estimator}_0.3"]
    #     mean_val_07:float = row[f"{col_name_conf_estimator}_0.7"]
    #
    #     new_row_value = (
    #         format_value_depending_on_whether_it_is_smallest(
    #             value=mean_val_03,
    #             is_smallest=col_name_conf_estimator == label_freq_to_smallest_est_map[0.3],
    #             float_precision=float_precision
    #         )
    #         + " / "
    #         + format_value_depending_on_whether_it_is_smallest(
    #             value=mean_val_07,
    #             is_smallest=col_name_conf_estimator == label_freq_to_smallest_est_map[0.7],
    #             float_precision=float_precision
    #         )
    #     )
    #     new_row.append(new_row_value)
    for col_name_conf_estimator in estimator_columns:
        mean_val_03:float = row[f"{col_name_conf_estimator}_0.3"]



        new_row_value_03 = format_value_depending_on_whether_it_is_smallest(
                value=mean_val_03,
                is_smallest=(
                        col_name_conf_estimator in label_freq_to_set_of_smallest_est_map[first_true_label_freq_to_include]
                ),
                float_precision=float_precision
        )
        new_row.append(new_row_value_03)


    for col_name_conf_estimator in estimator_columns:
        mean_val_07:float = row[f"{col_name_conf_estimator}_0.7"]
        new_row_value_07 = format_value_depending_on_whether_it_is_smallest(
                value=mean_val_07,
                is_smallest=(
                        col_name_conf_estimator in label_freq_to_set_of_smallest_est_map[second_true_label_freq_to_include]
                ),
                float_precision=float_precision

        )
        new_row.append(new_row_value_07)

    pretty_rows.append(new_row)


df_pretty: pd.DataFrame = pd.DataFrame(
    data=pretty_rows,
    columns=multi_index_columns
)
df_pretty.head()

$p$ \# R  $c=0.3$                                       \
                       CWA      PCA         IPW IPW ($-\Delta$)   
0    actedin    1   $50.4$   $39.8$  $\bm{0.2}$          $27.7$   
1    created    2  $475.6$   $42.6$  $\bm{4.1}$         $236.5$   
2  dealswith    7  $178.7$   $95.5$  $\bm{2.7}$         $109.2$   
3     diedin    1   $87.8$  $109.3$  $\bm{0.6}$          $38.7$   
4   directed    2  $561.3$   $61.1$  $\bm{6.7}$         $347.9$   

                   $c=0.7$                                       \
  IPW ($+\Delta$)      CWA      PCA         IPW IPW ($-\Delta$)   
0           $6.3$    $8.7$  $141.2$  $\bm{0.1}$           $3.6$   
1          $65.8$   $85.1$  $110.8$  $\bm{1.6}$          $30.9$   
2          $22.1$   $34.0$    $7.1$  $\bm{0.5}$          $10.5$   
3          $12.9$   $15.7$   $84.1$  $\bm{0.1}$           $5.0$   
4          $66.6$  $106.3$  $281.9$  $\bm{0.6}$          $32.3$   

                   
  IPW ($+\Delta$)  
0           $1.3$  
1          $15.1$  
2           $6.4$  
3           $2.7$  
4          $19.2$

In [54]:
df_pretty: pd.DataFrame = df_pretty.sort_values(
    by=["$p$"]
)

df_pretty.head()



$p$ \# R  $c=0.3$                                       \
                       CWA      PCA         IPW IPW ($-\Delta$)   
0    actedin    1   $50.4$   $39.8$  $\bm{0.2}$          $27.7$   
1    created    2  $475.6$   $42.6$  $\bm{4.1}$         $236.5$   
2  dealswith    7  $178.7$   $95.5$  $\bm{2.7}$         $109.2$   
3     diedin    1   $87.8$  $109.3$  $\bm{0.6}$          $38.7$   
4   directed    2  $561.3$   $61.1$  $\bm{6.7}$         $347.9$   

                   $c=0.7$                                       \
  IPW ($+\Delta$)      CWA      PCA         IPW IPW ($-\Delta$)   
0           $6.3$    $8.7$  $141.2$  $\bm{0.1}$           $3.6$   
1          $65.8$   $85.1$  $110.8$  $\bm{1.6}$          $30.9$   
2          $22.1$   $34.0$    $7.1$  $\bm{0.5}$          $10.5$   
3          $12.9$   $15.7$   $84.1$  $\bm{0.1}$           $5.0$   
4          $66.6$  $106.3$  $281.9$  $\bm{0.6}$          $32.3$   

                   
  IPW ($+\Delta$)  
0           $1.3$  
1          $15.1$  
2           $6.4$  
3           $2.7$  
4          $19.2$

# To file

In [55]:
# dir_latex_table: str = os.path.join(
#     kbc_e_metrics_project_dir,
#     "paper_latex_tables",
#     'known_prop_scores',
#     'scar'
# )
#
# if not os.path.exists(dir_latex_table):
#     os.makedirs(dir_latex_table)

filename_latex_table: str = os.path.join(
    dir_latex_table,
    "confidence-error-table-scar-rerun-agg-per-p.tex"
)
filename_tsv_table: str = os.path.join(
    dir_latex_table,
    "confidence-error-table-scar-rerun-agg-per-p.tsv"
)


with open(filename_latex_table, "w") as latex_ofile:
    with pd.option_context("max_colwidth", 1000):
        latex_ofile.write(
            df_pretty.to_latex(
                column_format="lr|llllll|llllll",
                index=False,
                float_format="{:0.3f}".format,
                escape=False,
            #     caption="$[widehat{conf}-conf]^2$ for SCAR. "
            #             "std=standard confidence, "
            #             "PCA (S) = PCA confidence with $s$ as domain, "
            #             "PCA (O) = PCA confidence with $o$  as domain, "
            #             "IPW = PCA confidence with $\hat{e}=e$, "
            #             "IPW +/- $" + f"{label_frequency_est_diff:0.1}" + "$ = IPW confidence with $\hat{e}=e+/-" +  f"{label_frequency_est_diff:0.1}" + "$."
            )
        )

with open(filename_tsv_table, "w") as tsv_ofile:
    tsv_ofile.write(df_pretty.to_csv(
        index=False,
        sep="\t"
    ))

print(filename_latex_table)


/home/joschout/Documents/Repos/KUL-PUL/paper_latex_tables/known_prop_scores/scar/confidence-error-table-scar-rerun-agg-per-p.tex
